In [1]:
import numpy as np
import concrete.numpy as cnp
from functools import reduce

In [3]:
FHE_SERVER_PATH = "server.zip"

fhe_server = cnp.Server.load(FHE_SERVER_PATH)

In [40]:
def generate_rotate_shift_rightx(x, globals_=None, locals_=None):
    function_body = f"""def rotate_shift_right{x}(a):
    return np.concatenate((a[-{x}:], a[:-{x}])).astype('int64')"""
    exec(function_body, globals_, locals_)

In [41]:
def generate_rotate_shift_leftx(x, globals_=None, locals_=None):
    function_body = f"""def rotate_shift_left{x}(a):
    return np.concatenate((a[{x}:], a[:{x}])).astype('int64')"""
    exec(function_body, globals_, locals_)

In [42]:
shift = 2

In [43]:
for s in range(1, shift + 1):
    generate_rotate_shift_rightx(s, globals(), locals())
    generate_rotate_shift_leftx(s, globals(), locals())

In [44]:
a = np.array([1, 2, 3, 4])
b = np.array([3, 2, 1, 1])

In [45]:
rotate_shift_right1(a)

array([4, 1, 2, 3])

In [46]:
rotate_shift_left1(a)

array([2, 3, 4, 1])

In [47]:
def hamming_distance(x, y) -> int:
    return np.sum(x ^ y)

In [48]:
def min_scalar(x, y):
    return (x + y - abs(x - y)) // 2

In [49]:
def min_array(a):
    return reduce(min_scalar, a)

In [50]:
def best_shifted_hamming_distance(x, y):
    h = hamming_distance(x, y)

    l1 = rotate_shift_left1(x)
    l2 = rotate_shift_left2(x)

    hl1 = hamming_distance(l1, y)
    hl2 = hamming_distance(l2, y)

    r1 = rotate_shift_right1(x)
    r2 = rotate_shift_right2(x)

    hr1 = hamming_distance(r1, y)
    hr2 = hamming_distance(r2, y)

    return min_array([h, hr1, hr2, hl1, hl2])

In [51]:
compiler = cnp.Compiler(
    best_shifted_hamming_distance, {"x": "encrypted", "y": "encrypted"}
)

In [52]:
inputset = [(a, b)]

In [53]:
circuit = compiler.compile(inputset)

In [54]:
print(circuit)

 %0 = x                              # EncryptedTensor<uint3, shape=(4,)>        ∈ [1, 4]
 %1 = y                              # EncryptedTensor<uint2, shape=(4,)>        ∈ [1, 3]
 %2 = bitwise_xor(%0, %1)            # EncryptedTensor<uint3, shape=(4,)>        ∈ [0, 5]
 %3 = sum(%2)                        # EncryptedScalar<uint4>                    ∈ [9, 9]
 %4 = %0[1:]                         # EncryptedTensor<uint3, shape=(3,)>        ∈ [2, 4]
 %5 = %0[:1]                         # EncryptedTensor<uint1, shape=(1,)>        ∈ [1, 1]
 %6 = concatenate((%4, %5))          # EncryptedTensor<uint3, shape=(4,)>        ∈ [1, 4]
 %7 = astype(%6, dtype=int_)         # EncryptedTensor<uint3, shape=(4,)>        ∈ [1, 4]
 %8 = %0[2:]                         # EncryptedTensor<uint3, shape=(2,)>        ∈ [3, 4]
 %9 = %0[:2]                         # EncryptedTensor<uint2, shape=(2,)>        ∈ [1, 2]
%10 = concatenate((%8, %9))          # EncryptedTensor<uint3, shape=(4,)>        ∈ [1, 4]
%11 = asty

In [55]:
# Clear execution
best_shifted_hamming_distance(a, b)

7

In [56]:
# FHE execution
circuit.encrypt_run_decrypt(a, b)

7

In [57]:
exec?

Signature: exec(source, globals=None, locals=None, /)
Docstring:
Execute the given source in the context of globals and locals.

The source may be a string representing one or more Python statements
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
Type:      builtin_function_or_method